In [22]:
# -*- coding: utf-8 -*-
"""
Created on Mon Feb 26 12:44:12 2024

@author: A200129952
"""

import sys
# from python_environment_check import check_packages
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from os.path import expanduser as ospath
import csv
import re
import locale
from datetime import datetime
from datetime import time



def replace_non_english(text):
    # Mapping of non-English letters to their English equivalents
    non_english_mapping = {
        'á': 'a', 'é': 'e', 'í': 'i', 'ó': 'o', 'ö': 'o', 'ő': 'o',
        'ú': 'u', 'ü': 'u', 'ű': 'u', 'Á': 'A', 'É': 'E', 'Í': 'I',
        'Ó': 'O', 'Ö': 'O', 'Ő': 'O', 'Ú': 'U', 'Ü': 'U', 'Ű': 'U',
        'Č': 'C', 'č': 'c', 'Ď': 'D', 'ď': 'd', 'Ě': 'E', 'ě': 'e',
        'Ň': 'N', 'ň': 'n', 'Ř': 'R', 'ř': 'r', 'Š': 'S', 'š': 's',
        'Ť': 'T', 'ť': 't', 'Ů': 'U', 'ů': 'u', 'Ý': 'Y', 'ý': 'y',
        'Ž': 'Z', 'ž': 'z'
    }
    
    # Replace non-English letters with English equivalents
    english_text = ''.join(non_english_mapping.get(char, char) for char in text)
    
    return english_text


def read_first_rows(csv_file,number):
    with open(csv_file, 'r',encoding='windows-1252') as file:
        reader = csv.reader(file, delimiter='\t')
        rows = [next(reader) for _ in range(number)]
        return rows

def add_columns_to_dataframe(input_file, *additional_columns):
    # Read the CSV file into a DataFrame
    df00 = pd.DataFrame()
    df00 = pd.read_csv(input_file,sep=';', encoding='utf-16', header=None)
    df01 = df00.iloc[1:, 2:]
    
    # Add additional columns to the beginning of the DataFrame
    for i, col in enumerate(additional_columns):
        df01.insert(i, f'Column{i+1}', col)
    return df01

def remove_non_alphanumeric(input_string):
    # Use regular expression to remove non-alphanumeric characters
    cleaned_string = re.sub(r'[^a-zA-Z0-9]', '', input_string)
    return cleaned_string

def remove_non_hungarian_alphanumeric(input_string):
    # Define regular expression pattern to match Hungarian alphanumeric characters
    pattern = r'[^a-zA-Z0-9áéíóöőúüűÁÉÍÓÖŐÚÜŰ_]'
    
    # Use regular expression to remove non-Hungarian alphanumeric characters
    cleaned_string = re.sub(pattern, '', input_string)
    return cleaned_string

def process_row(row, maxtime):
    time_part = row.when.split()[-1]
    date_first = row.frm.strftime('%Y-%m-%d %H:%M:%S').split()[0]
    date_second = row.to.strftime('%Y-%m-%d %H:%M:%S').split()[0]
    if maxtime <= time_part:
        maxtime = time_part
        row.when = date_first + ' ' + time_part
    else:
        maxtime = time(23, 59).strftime('%Y-%m-%d %H:%M:%S').split()[-1]
        row.when = date_second + ' ' + time_part
    return row, maxtime


def kutyak_alapadatai():

    path0=r'~/SCS/03AI/konyv/FlyBall/DataE/F7/'
    kuty = 'Wild_Runners_Flyball_Team_kutyak_alapadatai.xlsx'
    s = path0 + kuty
    print('From local path:', kuty)
    df1 = pd.read_excel(ospath(s), sheet_name=None)
    #for key in df.keys():
    for key, value in df1.items():
        df1[key].to_csv(path0 +'%s.csv' % key, sep=';', encoding='utf-16')
        df01 = pd.read_csv(path0 +'%s.csv' % key,sep=';',  encoding='utf-16') 
        df02 = df01.iloc[:, 1:]
    
    df02.to_csv('07.kutyak_alapadatai.csv', sep=';', index=False)        


def EFC_versenyadatok():

    path1=r"~\SCS\03AI\konyv\FlyBall\DataE\F7"
    #c:\SCS\03MI\proj\005.MT.1.haziverseny\FlyBall03\EFC_versenyadatokM.xlsm 
    #path1=r'c:/SCS/03MI/proj/005.MT.1.haziverseny/FlyBall03/'
    #path1=r'c:/SCS/03MI/proj/005.MT.1.haziverseny/FlyBall03/'
    path = os.getcwd() + '\\'
    print('path:', path)
    s = path + 'EFC_versenyadatokM.xlsm'
    #s = 'EFC_versenyadatokM.xlsm'
    print('From local path:', s)

    df2 = pd.read_excel(ospath(s), sheet_name=None)
    #for key in df.keys():
    months_hun = ["január", "február", "március", "április", "május", "június", "július", "augusztus", "szeptember", "október", "november", "december"]
    months_eng = ["january", "february", "march", "april", "may", "june", "july", "august", "september", "october", "november", "december"]
    rdf = pd.DataFrame()
    dfl = []
    
    first =  True 
    for key, value in df2.items():
        
        csv_file = path + '%s.csv'  % key
        df2[key].to_csv(csv_file, sep=';', encoding='utf-16')
        #df2[key].to_csv(csv_file, sep=';', encoding='windows-1250')
        first_rows = read_first_rows(csv_file, 1)
        for rows in first_rows:
            sepl = re.split(r'[;()]+', rows[0])
            hun_str = remove_non_hungarian_alphanumeric(sepl[4])
            eng_str = []
            for hun, eng in zip(months_hun, months_eng):
               if hun in hun_str and hun_str.index(hun) > 0:
                   date_str = hun_str.replace(hun, eng)
            date_object = datetime.strptime(date_str, "%Y%B%d")
        additional_columns = [key, remove_non_hungarian_alphanumeric(sepl[3]), date_object, date_object]        
        df01 = add_columns_to_dataframe(csv_file, *additional_columns)
        df01 = df01.rename(columns={
        df01.columns[0]:  'run',
        df01.columns[1]:  'place',
        df01.columns[2]:  'frm',
        df01.columns[3]:  'to',
        df01.columns[4]:  'division',
        df01.columns[5]:  'when',
        df01.columns[6]:  'who',
        df01.columns[7]:  'withwho',
        df01.columns[8]:  'total',
        df01.columns[9]:  'WLT',
        df01.columns[10]: 'hurdless',
        df01.columns[11]: 'name.1',
        df01.columns[12]: 'start.1',
        df01.columns[13]: 'dog.1',
        df01.columns[14]: 'name.2',
        df01.columns[15]: 'start.2',
        df01.columns[16]: 'dog.2',
        df01.columns[17]: 'name.3',
        df01.columns[18]: 'start.3',
        df01.columns[19]: 'dog.3',
        df01.columns[20]: 'name.4',
    	df01.columns[21]: 'start.4',
    	df01.columns[22]: 'dog.4',
        df01.columns[23]: 'colour',
        })
        #for column_name in df01.columns:
        #    print(column_name)
        maxtime = time(0, 0).strftime('%Y-%m-%d %H:%M:%S').split()[-1]  
        df02 = df01.iloc[1:]
        for index, row in df02.iterrows():
          if pd.isna(row['when']):  # Check if 'when' contains NaN
             df02 = df02.drop(index)  # Delete the row with NaN in 'when' column
          else:  
            time_part = row['when'].split()[-1]
            date_first = row['frm'].strftime('%Y-%m-%d %H:%M:%S').split()[0]
            date_second = row['to'].strftime('%Y-%m-%d %H:%M:%S').split()[0]
            if maxtime <= time_part:
               maxtime = time_part
               df02.loc[index, 'when'] = date_first + ' ' + time_part
            else:
               maxtime = time(23, 59).strftime('%Y-%m-%d %H:%M:%S').split()[-1]
               df02.loc[index, 'when'] = date_second + ' ' + time_part        
        df03 = df02.drop(columns=['frm','to'])
        dfl.append(df03)
        tdf = pd.concat( dfl,ignore_index=True)
    tdf.to_csv('07.EFC_versenyadatok.csv', sep=';', index=False)    


def FLYBALLCZ_versenyadatok():

    #path1=r'~/SCS/03AI/konyv/FlyBall/DataE/F4/'
    #c:\SCS\03MI\proj\005.MT.1.haziverseny\FlyBall03\EFC_versenyadatokM.xlsm 
    #path1 = r"~SCS\03AI\konyv\FlyBall\DataE"
    #path1=r'c:/SCS/03MI/proj/005.MT.1.haziverseny/FlyBall03/'
    path = os.getcwd() + '\\'
    #s ='FLYBALLCZ_versenyadatokM.xlsm'
    s = path + 'FLYBALLCZ_versenyadatokM.xlsm'
    print('From local path:', s)

    df2 = pd.read_excel(ospath(s), sheet_name=None)
    #for key in df.keys():
    months_hun = ["január", "február", "március", "április", "május", "június", "július", "augusztus", "szeptember", "október", "november", "december"]
    months_eng = ["january", "february", "march", "april", "may", "june", "july", "august", "september", "october", "november", "december"]
    rdf = pd.DataFrame()
    dfl = []
    
    
    first =  True 
    for key, value in df2.items():
        
        csv_file = path + '%s.csv'  % key
        df2[key].to_csv(csv_file, sep=';', encoding='utf-16')
        #df2[key].to_csv(csv_file, sep=';', encoding='windows-1250')
        first_rows = read_first_rows(csv_file, 1)
        for row in first_rows:
    #        print(row)
            sepl = re.split(r'[;()\-]+', row[0])
            hun_str = remove_non_hungarian_alphanumeric(sepl[4])
            #hun_str = replace_non_english(sepl[4])
    #        hun_str = sepl[4].replace(" ", "").replace(".", "")
            eng_str = []
            for hun, eng in zip(months_hun, months_eng):
               if hun in hun_str and hun_str.index(hun) > 0:
                   date_str1 = hun_str.replace(hun, eng)
            ending_numeric_chars =  re.findall(r'\d+$', date_str1)     
            date_str2 = date_str1.replace(''.join(ending_numeric_chars),remove_non_hungarian_alphanumeric(sepl[5]))
            #date_str2 = date_str1.replace(''.join(ending_numeric_chars),replace_non_english(sepl[5]))
            date_object1 = datetime.strptime(date_str1, "%Y%B%d")
            date_object2 = datetime.strptime(date_str2, "%Y%B%d")
        #additional_columns = [key, remove_non_hungarian_alphanumeric(sepl[3]), date_object1, date_object2]  
        additional_columns = [replace_non_english(key), replace_non_english(sepl[3]), date_object1, date_object2]    
        df01 = add_columns_to_dataframe(csv_file, *additional_columns)
        df01 = df01.rename(columns={
        df01.columns[0]:  'run',
        df01.columns[1]:  'place',
        df01.columns[2]:  'frm',
        df01.columns[3]:  'to',
        df01.columns[4]:  'division',
        df01.columns[5]:  'when',
        df01.columns[6]:  'who',
        df01.columns[7]:  'withwho',
        df01.columns[8]:  'total',
        df01.columns[9]:  'WLT',
        df01.columns[10]: 'hurdless',
        df01.columns[11]: 'name.1',
        df01.columns[12]: 'start.1',
        df01.columns[13]: 'dog.1',
        df01.columns[14]: 'name.2',
        df01.columns[15]: 'start.2',
        df01.columns[16]: 'dog.2',
        df01.columns[17]: 'name.3',
        df01.columns[18]: 'start.3',
        df01.columns[19]: 'dog.3',
        df01.columns[20]: 'name.4',
    	df01.columns[21]: 'start.4',
    	df01.columns[22]: 'dog.4',
    	df01.columns[23]: 'colour'    
        })
        #for column_name in df01.columns:
        #    print(column_name)
        maxtime = time(0, 0).strftime('%Y-%m-%d %H:%M:%S').split()[-1]  
        df02 = df01.iloc[1:]
        for index, row in df02.iterrows():
          if pd.isna(row['when']):  # Check if 'when' contains NaN
             df02 = df02.drop(index)  # Delete the row with NaN in 'when' column
          else:  
            time_part = row['when'].split()[-1]
            date_first = row['frm'].strftime('%Y-%m-%d %H:%M:%S').split()[0]
            date_second = row['to'].strftime('%Y-%m-%d %H:%M:%S').split()[0]
            if maxtime <= time_part:
               maxtime = time_part
               df02.loc[index, 'when'] = date_first + ' ' + time_part
            else:
               maxtime = time(23, 59).strftime('%Y-%m-%d %H:%M:%S').split()[-1]
               df02.loc[index, 'when'] = date_second + ' ' + time_part     
        df03 = df02.drop(columns=['frm','to'])
        dfl.append(df03)
        tdf = pd.concat( dfl,ignore_index=True)
    tdf.to_csv('07.FLYBALLCZ_versenyadatok.csv', sep=';',encoding='utf-8', index=False)   


In [23]:
kutyak_alapadatai()
EFC_versenyadatok()
FLYBALLCZ_versenyadatok()

From local path: Wild_Runners_Flyball_Team_kutyak_alapadatai.xlsx
path: C:\Users\A200129952\SCS\03AI\konyv\FlyBall\DataE\F7\
From local path: C:\Users\A200129952\SCS\03AI\konyv\FlyBall\DataE\F7\EFC_versenyadatokM.xlsm
From local path: C:\Users\A200129952\SCS\03AI\konyv\FlyBall\DataE\F7\FLYBALLCZ_versenyadatokM.xlsm
